이번에는 document chain에 대해 알아보자.

지금은 off-the-shelf chain들을 사용할거다.

그 다음에 우리만의 chain을 LangChain Expression Language으로 만들거다.

지금은 LCEL이 대세!!





다양한 type의 document chain 생성방식 중 먼저 우리가 사용해 볼 것은 'stuff'이다.

우리가 찾은 document들로 prompt를 stuff(채우기)하는데 사용한다는 뜻이다.

1. 먼저 질문을 한다. "what is foo?"
1. 그 질문을 사용해서 document를 search(검색)할 거다.
1. 그 다음 찾은 document들을 Prompt에 입력해서 model에게 전달한다.
1. 그리고 model은 입력된 질문과 documents를 토대로 우리에게 답변해줄 거다.

여기엔 이미 우리를 위해 만들어져 있는 (off-the-shelf) chain이 있다.

---

RetrievalQA를 불러오자.

일단 출력 결과를 한번 보고나서 직접 코드를 작성해서 구현해보자.


```python
chain = RetrievalQA.from_chain_type()
```
이건 일종의 Constructor(생성자함수)인데, LLM을 입력값으로 받는다.

chain_type 도 입력가능하다. default='stuff'

그리고 RetrievalQA chain은 retriever도 입력값으로 요구한다.



retriever는 무엇일까?

문서에 따르면 retriever는 class의 interface이다.

document를 많은 장소로부터 Retrieve(선별하여 가져오기) 할 수 있다.

vector store 말고 다른데서도!

이번에는 vector store를 가져와서 Retriever로 만들어보자.

```python
retriever=vectorstore.as_retriever(),
```

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

# llm
llm = ChatOpenAI()
# 캐시 경로
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
# .from_bytes_store 메서드는 임베딩 작업을 위해 필요한 embedder의 입력을 요구한다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = Chroma.from_documents(docs, embeddings)

# chain 구현
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

이제 이 chain을 실행(run)해보자.

In [ ]:
chain.run("Where does Winston Smith live?")

In [ ]:
chain.run("Describe Victory Mansions?")

Chroma가 아닌 다른 vectorstores인 FAISS 를 사용해보자.

한 줄만 변경해도 적용할 수 있다.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

# llm
llm = ChatOpenAI()
# 캐시 경로
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
# .from_bytes_store 메서드는 임베딩 작업을 위해 필요한 embedder의 입력을 요구한다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# vectorstore 변경
vectorstore = FAISS.from_documents(docs, embeddings)

# chain 구현: chain_type 변경
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

# Recap

먼저 이번 섹션에서 했던 첫 번째 작업은 바로 file들을 load하는 거였다.

text_loader를 이용해서 file들을 load하는 방법을 배움.

그리고 UnstructuredFileLoader에 대해서 살펴봤다.

UnstructuredFileLoader로 다양한 file들을 load할 수 있다.

그 다음 file을 분할(split)하는 방법도 배웠다. 아주 긴 text를 다루기 위해서

또 그 긴 text를 작은 document들로 나누고자 했다.

거대한 단일 document보다는 작은 여러개를 LLM에게 전달할 때 검색 성능이 더 좋아지기 떄문에.

작업을 요약하면, document를 적재(load)하고 분할(split)했다.

---

그리고 또 embedding에 대해 배웠다.

임베딩은 text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현했다.

우리는 OpenAIEmbeddings model을 사용했다.

그리고 CacheBacedEmbedding을 사용하여 만들어진 Embedding을 cache(저장)했다.

CacheBacedEmbedding에 embeddings model을 전달하고, 데이터가 cache될 directory(폴더)를 지정했다.

그 다음으로는 Vector store를 호출했다. 마지막에는 FAISS

document와 embedding와 함께 .from_documents 메서드를 호출했다.

이 메서드는 document별로 embedding 작업 후 결과를 저장한 vector store를 반환하는데

이를 이용해 document 검색도 하고, 연관성이 높은 document들을 찾기도 했다.

---

마지막으로 RetrievalQA라는 Chain.

필요한 입력값은 llm, chain의 type, 그리고 retriever.


```python
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

```
retriever는 Langchain이 제공하는 class 또는 interface의 일종인데,

document를 검색해서 찾아오는(retrieve)기능을 가지고 있다.

as_retriever() 메서드 호출만으로 vector store 을 retriever로 전환할 수 있었다.


